---
---

# Modelling

---
---

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import sys

sys.path.append("../src")
from load_config import load_constants_from_yaml

Define constants

In [15]:
constants = load_constants_from_yaml("../constants.yml")

SAMPLING_RATING = constants["SAMPLING_RATING"]
FRAME_LENGTH_ENERGY = constants["FRAME_LENGTH_ENERGY"]
THRESHOLD_PERCENTAGE = constants["THRESHOLD_PERCENTAGE"]
MIN_SILENCE_DURATION = constants["MIN_SILENCE_DURATION"]
HOP_LENGTH = constants["HOP_LENGTH"]
TEST_SIZE = 0.2
FIRST_LAYER_NEURONS = 128
SECOND_LAYER_NEURONS = 64
RANDOM_STATE = 42
processed_data_path = "../data/processed/"

In [16]:
EPOCHS = 15
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
THRESHOLD_CLASSIFICATION = 0.5

Load data

In [17]:
df = pd.read_csv(processed_data_path+"df_transformed.csv")

In [18]:
df.head(5)

,label,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13
0,1,-2.199065,-0.881256,0.591127,-0.169963,0.582260,0.748747,1.134152,0.801761,0.953608,0.726335,0.872313,0.664804,0.865082
1,1,-2.268200,-1.028002,0.353531,-0.367497,0.426105,0.683014,1.162779,0.914434,1.118985,0.903471,1.027281,0.782093,0.921110
2,1,-2.359687,-1.228744,-0.007906,-0.736143,0.002961,0.278296,0.772721,0.561240,0.813884,0.648693,0.813558,0.586497,0.760301
3,1,-2.359693,-1.228758,-0.007932,-0.736172,0.002924,0.278254,0.772673,0.561186,0.813826,0.648631,0.813491,0.586419,0.760223
4,1,-2.359697,-1.228765,-0.007944,-0.736186,0.002906,0.278235,0.772650,0.561161,0.813798,0.648601,0.813459,0.586382,0.760185


Split data set

In [19]:
X = df.drop("label", axis=1)
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_STATE
)

In [20]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [21]:
class CSVDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        return self.X_data[idx], self.y_data[idx]

In [22]:
# Create dataset instances for training and testing
train_dataset = CSVDataset(X_train_tensor, y_train_tensor)
test_dataset = CSVDataset(X_test_tensor, y_test_tensor)

# Create DataLoader to batch and shuffle the data
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE)

### Model architecture

Define the model

In [23]:
class Net(nn.Module):
    def __init__(self, input_shape, first_layer_neurons, second_layer_neurons):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_shape, first_layer_neurons)
        self.fc2 = nn.Linear(first_layer_neurons, second_layer_neurons)
        self.output = nn.Linear(second_layer_neurons, 1)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.output(x))
        return x

In [24]:
input_shape = X_train.shape[1]
net = Net(
    input_shape,
    first_layer_neurons=FIRST_LAYER_NEURONS,
    second_layer_neurons=SECOND_LAYER_NEURONS,
)

In [25]:
criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

### Train the model

In [26]:
for epoch in range(EPOCHS):
    running_loss = 0.0
    for inputs, labels in train_loader:
        labels.unsqueeze_(-1)
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

print("Training finished.")

Epoch 1, Loss: 0.4696
Epoch 2, Loss: 0.4184
Epoch 3, Loss: 0.3958
Epoch 4, Loss: 0.3802
Epoch 5, Loss: 0.3694
Epoch 6, Loss: 0.3614
Epoch 7, Loss: 0.3549
Epoch 8, Loss: 0.3493
Epoch 9, Loss: 0.3450
Epoch 10, Loss: 0.3416
Epoch 11, Loss: 0.3380
Epoch 12, Loss: 0.3349
Epoch 13, Loss: 0.3324
Epoch 14, Loss: 0.3306
Epoch 15, Loss: 0.3281
Training finished.


### Evaluation

In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [28]:
def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            labels.unsqueeze_(-1)
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # apply a threshold to outputs
            predicted = (outputs > THRESHOLD_CLASSIFICATION).float()

            # Collect predictions and labels for metrics
            all_predictions.extend(predicted.cpu().numpy().flatten())
            all_labels.extend(labels.cpu().numpy().flatten())

            # correct predictions count
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    # Calculate average loss and accuracy
    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total * 100  # percentage

    # Calculate precision, recall, F1 score
    precision = precision_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)

    # Compute confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_predictions, normalize="all")

    return {
        "avg_loss": avg_loss,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix,
    }

In [29]:
def show_results(results):
    print(f"Average Loss: {results['avg_loss']:.4f}")
    print(f"Accuracy: {results['accuracy']:.2f}%")
    print(f"Precision: {results['precision']:.4f}")
    print(f"Recall: {results['recall']:.4f}")
    print(f"F1 Score: {results['f1_score']:.4f}")
    print(f"Confusion Matrix:\n{results['confusion_matrix']}")

#### Evaluate the model on training set

In [30]:
results = evaluate_model(net, train_loader, criterion)
show_results(results=results)

Average Loss: 0.3212
Accuracy: 85.77%
Precision: 0.8225
Recall: 0.8127
F1 Score: 0.8176
Confusion Matrix:
[[0.53884806 0.06881211]
 [0.07349657 0.31884326]]


#### Evaluate the model on the testing set

In [31]:
results = evaluate_model(net, test_loader, criterion)
show_results(results=results)

Average Loss: 0.3405
Accuracy: 84.83%
Precision: 0.8105
Recall: 0.8004
F1 Score: 0.8054
Confusion Matrix:
[[0.53424146 0.07342094]
 [0.07831449 0.31402312]]


### Serialize the model

In [33]:
torch.save(net.state_dict(), "../models/net_weights.pth")